In [3]:
from pyspark.sql import SparkSession

# Khởi động SparkSession và thiết lập mức log
spark = SparkSession.builder \
    .appName("AddStudentDataToHDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

# Thiết lập mức log để giảm thiểu cảnh báo
spark.sparkContext.setLogLevel("ERROR")

# Tạo một DataFrame mẫu mới về sinh viên
student_data = [("John", "Doe", "123456", "Computer Science"),
                ("Jane", "Smith", "654321", "Mathematics"),
                ("Emily", "Jones", "789012", "Physics")]

student_columns = ["firstname", "lastname", "student_id", "major"]

student_df = spark.createDataFrame(student_data, schema=student_columns)

# Ghi DataFrame mới vào HDFS vào thư mục khác
student_df.write.csv("hdfs://namenode:9000/user/nhatrinh/student_data.csv", header=True)

# Dừng SparkSession
spark.stop()


In [5]:
from pyspark.sql import SparkSession

# Khởi động SparkSession và thiết lập mức log
spark = SparkSession.builder \
    .appName("ReadAllDataFromHDFS") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

# Thiết lập mức log để giảm thiểu cảnh báo
spark.sparkContext.setLogLevel("ERROR")

# Đọc dữ liệu từ file cũ
df_old = spark.read.csv("hdfs://namenode:9000/user/nhatrinh/example.csv", header=True, inferSchema=True)

# Đọc dữ liệu từ file mới
df_new = spark.read.csv("hdfs://namenode:9000/user/nhatrinh/new_data.csv", header=True, inferSchema=True)

# Đọc dữ liệu từ file sinh viên
df_student = spark.read.csv("hdfs://namenode:9000/user/nhatrinh/student_data.csv", header=True, inferSchema=True)

# Hiển thị dữ liệu kết hợp (nếu cần thiết, có thể kết hợp dữ liệu thành một DataFrame duy nhất)
df_all = df_old.union(df_new).union(df_student)

# Hiển thị dữ liệu kết hợp
df_all.show()

# Dừng SparkSession
spark.stop()


+---------+--------+-------+----------------+
|firstname|lastname|country|           state|
+---------+--------+-------+----------------+
|   Robert|Williams|    USA|              CA|
|  Michael|    Rose|    USA|              NY|
|    James|   Smith|    USA|              CA|
|    Maria|   Jones|    USA|              FL|
|     Emma|  Watson| France|           Paris|
|    Alice|   Brown| Canada|              ON|
|     John|     Doe|     UK|          London|
|     John|     Doe| 123456|Computer Science|
|     Jane|   Smith| 654321|     Mathematics|
|    Emily|   Jones| 789012|         Physics|
+---------+--------+-------+----------------+

